In [ ]:
from itertools import chain
import sqlite3
import pandas as pd
from skyscanner import FlightsCache

DB_SCHEMA = """
PRAGMA foreign_keys = ON;

CREATE TABLE place
  ( id     INTEGER          PRIMARY KEY
  , name   TEXT    NOT NULL
  , type   TEXT    NOT NULL
  );

CREATE TABLE quote
  ( user_id                    TEXT
  , quote_id                   INTEGER
  , direct                     BOOLEAN  NOT NULL
  , minimum_price              FLOAT    NOT NULL
  , quote_datetime             DATETIME NOT NULL
  , outbound_departure_date    DATETIME NOT NULL
  , outbound_origin_id         INTEGER  NOT NULL REFERENCES place(id)
  , outbound_destination_id    INTEGER  NOT NULL REFERENCES place(id)
  , outbound_carriers          TEXT     NOT NULL
  , inbound_departure_date     DATETIME NOT NULL
  , inbound_origin_id          INTEGER  NOT NULL REFERENCES place(id)
  , inbound_destination_id     INTEGER  NOT NULL REFERENCES place(id)
  , inbound_carriers           TEXT     NOT NULL
  
  , PRIMARY KEY (user_id, quote_id)
  );

"""

sqlite3.enable_callback_tracebacks(True)

db = sqlite3.connect(':memory:')
db.executescript(DB_SCHEMA)

service = FlightsCache('se893794935794863942245517499220')
params = dict( market='US'
             , currency='USD'
             , locale='en-US'
             , destinationplace='US'
             , outbounddate='2016-08'
             , inbounddate='2016-08'
             )

user_params = { 'byron': dict(originplace='DTW-sky')
              , 'tara': dict(originplace='SFO-sky')
              }

for user in user_params:
    p = params.copy()
    p.update(user_params[user])
    results = service.get_cheapest_quotes(**p).parsed
    places = [(d['PlaceId'], d['Name'], d['Type'])
              for d in results['Places']]
    db.executemany('INSERT OR REPLACE INTO place VALUES (?,?,?)', set(places))
    quotes = [(user, d['QuoteId'], d['Direct'], d['MinPrice'], d['QuoteDateTime'],
               d['OutboundLeg']['DepartureDate'], d['OutboundLeg']['OriginId'],
               d['OutboundLeg']['DestinationId'], str(d['OutboundLeg']['CarrierIds']),
               d['InboundLeg']['DepartureDate'], d['InboundLeg']['OriginId'],
               d['InboundLeg']['DestinationId'], str(d['InboundLeg']['CarrierIds']))
              for d in results['Quotes']]
    db.executemany('INSERT INTO quote VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?)', quotes)

place_table = pd.read_sql('SELECT * FROM place', con=db)
place_map = dict(zip(place_table.id, place_table.name))

In [ ]:
query = """
SELECT
    SUM(minimum_price) AS minimum_total_price
  , COUNT(*) AS number_of_people
  , outbound_departure_date AS start_date
  , inbound_departure_date AS end_date
  , place.name AS place_name
FROM place
JOIN quote ON place.id = quote.outbound_destination_id

GROUP BY place.id, inbound_departure_date, outbound_departure_date

"""

data = pd.read_sql(query, con=db, parse_dates=['start_date', 'end_date'])
data[data.number_of_people == 2].sort_values('minimum_total_price')